In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("Joins_Lab").getOrCreate()

# Таблица пользователей (User Dimension)
users_data = [
    (1, "Alice", "USA"),
    (2, "Bob", "UK"),
    (3, "Charlie", "CN"),
    (4, "David", "USA")
]

users_df = spark.createDataFrame(users_data, ["id", "name", "country"])

# Таблица заказов (Orders Fact)
orders_data = [
    (101, 1, 500.0), # Alice
    (102, 1, 300.0), # Alice
    (103, 2, 1200.0), # Bob
    (104, 99, 50.0)   # Неизвестный пользователь (id 99)
]

orders_df = spark.createDataFrame(orders_data, ["order_id", "user_id", "amount"])

users_df.show()
orders_df.show()

+---+-------+-------+
| id|   name|country|
+---+-------+-------+
|  1|  Alice|    USA|
|  2|    Bob|     UK|
|  3|Charlie|     CN|
|  4|  David|    USA|
+---+-------+-------+

+--------+-------+------+
|order_id|user_id|amount|
+--------+-------+------+
|     101|      1| 500.0|
|     102|      1| 300.0|
|     103|      2|1200.0|
|     104|     99|  50.0|
+--------+-------+------+



### Классический Inner Join

In [2]:
# Синтаксис: df1.join(df2, условие, тип)

# Вариант 1: Если имена колонок разные (id vs user_id)
# В результате будут и 'id', и 'user_id'
join_condition = users_df.id == orders_df.user_id
inner_joined = users_df.join(orders_df, join_condition, "inner")

inner_joined.show()
# Charlie и David исчезли (нет заказов).
# Заказ 104 исчез (нет пользователя).

+---+-----+-------+--------+-------+------+
| id| name|country|order_id|user_id|amount|
+---+-----+-------+--------+-------+------+
|  1|Alice|    USA|     101|      1| 500.0|
|  1|Alice|    USA|     102|      1| 300.0|
|  2|  Bob|     UK|     103|      2|1200.0|
+---+-----+-------+--------+-------+------+



### Left Join (Обогащение данных)

In [3]:
# Хотим ВСЕ заказы, даже если user_id не найден в базе
left_joined = orders_df.join(users_df, orders_df.user_id == users_df.id, "left")

left_joined.show()
# Заказ 104 остался, но name и country стали NULL.

+--------+-------+------+----+-----+-------+
|order_id|user_id|amount|  id| name|country|
+--------+-------+------+----+-----+-------+
|     101|      1| 500.0|   1|Alice|    USA|
|     102|      1| 300.0|   1|Alice|    USA|
|     103|      2|1200.0|   2|  Bob|     UK|
|     104|     99|  50.0|NULL| NULL|   NULL|
+--------+-------+------+----+-----+-------+



### Проблема дубликатов имен колонок (Ambiguous columns)

In [5]:
# Переименуем для примера
orders_renamed = orders_df.withColumnRenamed("user_id", "id")

# ОШИБКА НОВИЧКА:
# bad_join = users_df.join(orders_renamed, users_df.id == orders_renamed.id)
# bad_join.select("id") -> Упадет с ошибкой "Reference 'id' is ambiguous"

# РЕШЕНИЕ 1 (Автоматическое слияние):
# Передаем имя колонки строкой. Spark оставит только одну колонку 'id'.
smart_join = users_df.join(orders_renamed, "id", "inner")
smart_join.show()

# РЕШЕНИЕ 2 (Алиасы):
# Использовать .alias() для таблиц ПЕРЕД джойном, но это сложнее.

+---+-----+-------+--------+------+
| id| name|country|order_id|amount|
+---+-----+-------+--------+------+
|  1|Alice|    USA|     101| 500.0|
|  1|Alice|    USA|     102| 300.0|
|  2|  Bob|     UK|     103|1200.0|
+---+-----+-------+--------+------+



### Broadcast Join (Явная оптимизация)

In [6]:
# Функция broadcast помечает DataFrame как "маленький"
broadcast_join = orders_df.join(F.broadcast(users_df), orders_df.user_id == users_df.id, "left")

broadcast_join.explain()
# В плане вы увидите "BroadcastHashJoin" вместо "SortMergeJoin".
# Это признак высшего пилотажа.

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [user_id#7L], [id#0L], LeftOuter, BuildRight, false
   :- Scan ExistingRDD[order_id#6L,user_id#7L,amount#8]
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [plan_id=397]
      +- Filter isnotnull(id#0L)
         +- Scan ExistingRDD[id#0L,name#1,country#2]


